In [34]:
import torch
import pickle
import models
import dataset
import time

In [35]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

with open('./dist/resnet/extractor1d-170.pkl', 'rb') as f:
    extractor = pickle.load(f)

extractor.to(device)
extractor.eval()

Extractor(
  (resnet): Net1D(
    (first_conv): MyConv1dPadSame(
      (conv): Conv1d(1, 20, kernel_size=(16,), stride=(2,))
    )
    (first_bn): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (first_activation): Swish()
    (stage_list): ModuleList(
      (0): BasicStage(
        (block_list): ModuleList(
          (0): BasicBlock(
            (bn1): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (activation1): Swish()
            (do1): Dropout(p=0.5, inplace=False)
            (conv1): MyConv1dPadSame(
              (conv): Conv1d(20, 64, kernel_size=(1,), stride=(1,))
            )
            (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (activation2): Swish()
            (do2): Dropout(p=0.5, inplace=False)
            (conv2): MyConv1dPadSame(
              (conv): Conv1d(64, 64, kernel_size=(16,), stride=(2,), groups=4)
            )
          

In [36]:
extractor_resnet = extractor

In [37]:
with open('./dist/mlp/extractor1d-final.pkl', 'rb') as f:
    extractor = pickle.load(f)

extractor.to(device)
extractor.eval()

ExtractorMLP(
  (conv1d): MyConv1dPadSame(
    (conv): Conv1d(1, 1, kernel_size=(1,), stride=(1,))
  )
  (linear_in): Linear(in_features=20, out_features=65, bias=True)
  (backbone): Sequential(
    (linear-0): Linear(in_features=65, out_features=65, bias=True)
    (act-0): ReLU()
  )
  (linear_out): Linear(in_features=65, out_features=39, bias=True)
)

In [38]:
extractor_mlp = extractor

In [39]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [40]:
cnt_resnet_params = count_parameters(extractor_resnet)
cnt_resnet_params

5338790

In [41]:
cnt_mlp_params = count_parameters(extractor_mlp)
cnt_mlp_params

8231

In [42]:
def mem_consumption(cnt, gadge='GB'):
    # KB, MB, GB
    if gadge == 'KB':
        mul = 10
    elif gadge == 'MB':
        mul = 20
    else:
        gadge = 30

    mem_comsumption = {
        'f32': f"{cnt*4/2**mul:5}{gadge}",
        'f16': f"{cnt*2/2**mul:5}{gadge}",
        'i8': f"{cnt*1/2**mul:5}{gadge}",
    }

    return mem_comsumption

In [43]:
mem_consumption(cnt_mlp_params, gadge='KB')

{'f32': '32.15234375KB', 'f16': '16.076171875KB', 'i8': '8.0380859375KB'}

In [44]:
mem_consumption(cnt_resnet_params, gadge='MB')

{'f32': '20.365867614746094MB',
 'f16': '10.182933807373047MB',
 'i8': '5.091466903686523MB'}

## inference time

In [45]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import  train_test_split

data_files = [name for name in os.listdir('./data')]
dfs = []

for i in range(len(data_files)):
    fname = './data/' + data_files[i]
    df = pd.read_csv(fname)
    # df.columns = [x.strip().lstrip() for x in df.columns]

    dfs.append(df)

df = pd.concat(dfs, axis=0, ignore_index=True)

# Remove spaces in the front and the end of the column names for better human reading
df.columns = [x.lstrip().strip().replace('�', '-') for x in df.columns]

# replace inf values to nan
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)

df_train, df_val = train_test_split(df, stratify=df['Label'], test_size=0.2, random_state=42)

In [46]:
X = df_val.drop(['Label'], axis=1)
y = df_val['Label']

In [47]:
with open('./dist/labelencoder-resnet.pkl', 'rb') as f:
    label_encoder = pickle.load(f)

with open('./dist/preprocessor-resnet.pkl', 'rb') as f:
    preprocessor = pickle.load(f)

In [48]:
X = preprocessor.transform(X)
y = label_encoder.transform(y)

In [49]:
X = np.expand_dims(X, axis=2).transpose((0, 2, 1))

dataset = dataset.NumpyDataset(X, y)
len(dataset)

565576

In [50]:
from torch.utils.data import DataLoader

embeddings = []
labels = []
batch_size = 1024
dataloader = DataLoader(dataset=dataset, batch_size=batch_size)

start = time.time()

for X, y in dataloader:
    X = X.to(device)
    X_embedding = extractor_resnet(X)
    embeddings.append(X_embedding.cpu().detach().numpy())

    labels.append(y)

end = time.time()

print(f"elapsed {end - start}s")

elapsed 14.002958536148071s


In [51]:
from torch.utils.data import DataLoader

embeddings = []
labels = []
batch_size = 1024
dataloader = DataLoader(dataset=dataset, batch_size=batch_size)

start = time.time()

for X, y in dataloader:
    X = X.to(device)
    X_embedding = extractor_mlp(X)
    embeddings.append(X_embedding.cpu().detach().numpy())

    labels.append(y)

end = time.time()

print(f"elapsed {end - start}s")

elapsed 2.366269588470459s


## model visualization

In [54]:
import onnx
input_names = ["PCA"]
output_names = ["embedding"]

X_export = X[:128, :, :]

torch.onnx.export(extractor_mlp, X_export, "extractor_mlp.onnx", input_names=input_names, output_names=output_names)

================ Diagnostic Run torch.onnx.export version 2.0.1 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



/home/chan/Documents/CSI5388/project/resnet1d/net1d.py:56: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  p = max(0, (out_dim - 1) * self.stride + self.kernel_size - in_dim)
/home/chan/miniconda3/envs/csi5388/lib/python3.10/site-packages/torch/onnx/_internal/jit_utils.py:306: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally at /opt/conda/conda-bld/pytorch_1682343967769/work/torch/csrc/jit/passes/onnx/constant_fold.cpp:179.)
  _C._jit_pass_onnx_node_shape_type_inference(node, params_dict, opset_version)
/home/chan/miniconda3/envs/csi5388/lib/python3.10/site-packages/torch/onnx/utils.py:689: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >=

In [55]:
torch.onnx.export(extractor_resnet, X_export, "extractor_resnet.onnx", input_names=input_names, output_names=output_names)

/home/chan/Documents/CSI5388/project/resnet1d/net1d.py:56: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  p = max(0, (out_dim - 1) * self.stride + self.kernel_size - in_dim)
/home/chan/miniconda3/envs/csi5388/lib/python3.10/site-packages/torch/onnx/_internal/jit_utils.py:306: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally at /opt/conda/conda-bld/pytorch_1682343967769/work/torch/csrc/jit/passes/onnx/constant_fold.cpp:179.)
  _C._jit_pass_onnx_node_shape_type_inference(node, params_dict, opset_version)
/home/chan/miniconda3/envs/csi5388/lib/python3.10/site-packages/torch/onnx/utils.py:689: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >=

================ Diagnostic Run torch.onnx.export version 2.0.1 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

